# Association Rules from Cineplex Concessions

- Stephen W. Thomas
- October 2020
- For: MMA/GMMA/MMAI 869

This Notebook will find basic association rules.

In [1]:
import pandas as pd
import numpy as np

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

# Load in the data 

In [2]:
version_str = "_v1"
input_dir = "clean/"

In [3]:
items = pd.read_csv(input_dir+"items"+version_str+".csv", encoding = "ISO-8859-1")
visits_items = pd.read_csv(input_dir+"visit_items"+version_str+".csv", encoding = "ISO-8859-1")

In [4]:
# Don't need to consider any items with ID 0 (as that represents movie ticket purchases)
visits_items.shape
visits_items = visits_items[visits_items['CINEPLEX_Item_ID'] != 0]
visits_items.shape

(170060, 8)

(103134, 8)

In [5]:
# Let's merge with the items table in order to extract the item description
all_df = visits_items.merge(items, how="left", on="CINEPLEX_Item_ID")

In [6]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.float_format', lambda x: '%.1f' % x):
    display(all_df.head())

,Visit_ID,CINEPLEX_D_Membership_ID,CINEPLEX_Item_ID,Spend,Discount,Revenue,Points_Earned,Points_Redeemed,CINEPLEX_Item_Description,CINEPLEX_Item_Status,EIS_Group_Description,EIS_Sub_Group_Description,Item_Report_Code2,Item_Report_Code3,Item_Class_Description1,Item_Class_Description2,Item_Class_Description3,Is_Item_Package,Is_Box_Office_Item,US_Is_Butter,US_Is_Kiddietray,US_Is_Voucher,US_Is_LrgPopcorn,US_Is_Coffee,US_Is_Candy,US_Is_Alcohol,US_Is_MenuFood
0,48032,8518029,94119,5.8,0.6,5.8,29,0,VHO DRINK - VIP DRINK LARGE 32oz,A,0,0,0-350 Cals,0,VHO-DRINK,0,0,0,0,0,0,0,0,0,0,0,0
1,48032,8518029,93855,1.0,0.1,1.0,5,0,VHO CORN - BUTTER - SMALL,A,Popcorn,TOPPING,Add 60-320 Cals,0,VHO-POPCORN,0,0,0,0,0,0,0,0,0,0,0,0
2,48032,8518029,99025,6.5,0.7,6.5,33,0,VHO CORN - VIP POPCORN SMALL 85oz,A,0,0,400 Cals,0,VHO-POPCORN,0,0,0,0,0,0,0,0,0,0,0,0
3,52064,6767039,93853,1.0,0.1,1.0,5,0,VHO CORN - BUTTER - LARGE,A,Popcorn,TOPPING,Add 60-320 Cals,Y,VHO-POPCORN,0,0,0,0,1,0,0,0,0,0,0,0
4,52064,6767039,93853,1.0,0.1,1.0,5,0,VHO CORN - BUTTER - LARGE,A,Popcorn,TOPPING,Add 60-320 Cals,Y,VHO-POPCORN,0,0,0,0,1,0,0,0,0,0,0,0


In [7]:
# The mlxtend package wants the data in "wide" format, meaning: one row for every visit, and one 
# column for every possible item, and a 0/1 in each cell indicating if that item was in that visit

# Let's use some pandas magic to transform from long to wide format (there are probably other ways to do this)
trans = all_df[['Visit_ID', 'CINEPLEX_Item_Description']]
trans = pd.get_dummies(trans, prefix="", prefix_sep="")
trans = trans.groupby('Visit_ID').max()
trans.shape
trans.head()

(24466, 339)

,"*COMBO 1 - Lrg Corn, Lrg Drink, Lrg Candy","*COMBO 1 - Lrg Corn, Lrg Drink, Lrg Candy (FR)","*COMBO 2 - 2x Rg Corn, 2x Rg Drink, Lrg Candy","*COMBO 3 - Lrg Corn, 2x Rg Drink, Lrg Candy",4DX SEAT FEE,<Unknown>,<Unrecognized Item>,AXE THROW,BOWL WEEKDAY GAME,BP TAG+$15 - 90 CR,...,VHO XSCAPE - (I) PLAY CARD $25,VHO XSCAPE - PLAY CARD $20,VHO XSCAPE - PLAY CARD $25,VHO YF - SMOOTHY LRG,VHO YF - YOGURT LARGE,VHO YF - YOGURT SMALL,VHO YY - FULL SERVE YOBLEND,VHO YY - FULL SERVE YOBOWL LARGE,VHO YY - FULL SERVE YOBOWL REGULAR,VHO YY - YOBOWL (KG)
Visit_ID,,,,,,,,,,,,,,,,,,,,,
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Find Frequent Itemsets

mlxtend will use the apriori algorithm to find all the frequent itemsets. It will return them as a pandas DataFrame, so we can use regular pandas functions to slice and dice and explore.

In [8]:
%time frequent_itemsets = apriori(trans, min_support=0.001, use_colnames=True)

Wall time: 6.84 s


In [9]:
# Add a column that shows how many items are in the itemset
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2243 entries, 0 to 2242
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   support   2243 non-null   float64
 1   itemsets  2243 non-null   object 
 2   length    2243 non-null   int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 52.7+ KB


In [10]:
display(frequent_itemsets.head(10))

,support,itemsets,length
0,0.003065,"(*COMBO 1 - Lrg Corn, Lrg Drink, Lrg Candy)",1
1,0.002289,"(*COMBO 2 - 2x Rg Corn, 2x Rg Drink, Lrg Candy)",1
2,0.003229,"(*COMBO 3 - Lrg Corn, 2x Rg Drink, Lrg Candy)",1
3,0.001431,(4DX SEAT FEE),1
4,0.001267,(<Unknown>),1
5,0.002698,(<Unrecognized Item>),1
6,0.001431,(BP TAG+$30 - 210 CR),1
7,0.001471,(PREM TAG+$30 - 212 CR),1
8,0.001471,(TAG $30 - 200 CR),1
9,0.119104,(VHO AVX - Reserve Seating Fee),1


In [11]:
display(frequent_itemsets.tail(10))

,support,itemsets,length
2233,0.001308,"(VHO MERCH - UPSIZE TOPPER (ONLY), VHO DRINK - CUP KIDDIE FOUNTAIN, VHO CNDY - M&M MINI, VHO CORN - KIDDIE TRAY (SANS DRINK), VHO DRINK - CUP LARGE 32 OZ FOUNTAIN, VHO CORN - POPCORN LARGE 170 OZ)",6
2234,0.002207,"(VHO DRINK - CUP REGULAR 24 OZ FOUNTAIN, VHO DRINK - CUP KIDDIE FOUNTAIN, VHO CNDY - M&M MINI, VHO CORN - KIDDIE TRAY (SANS DRINK), VHO MERCH - PROMO CUP KIDDIE SURCHARGE, VHO CORN - POPCORN LARGE 170 OZ)",6
2235,0.002493,"(VHO DRINK - CUP REGULAR 24 OZ FOUNTAIN, VHO DRINK - CUP KIDDIE FOUNTAIN, VHO CNDY - M&M MINI, VHO CORN - KIDDIE TRAY (SANS DRINK), VHO MERCH - PROMO CUP KIDDIE SURCHARGE, VHO CORN - POPCORN REGULAR 130oz.)",6
2236,0.001104,"(VHO MERCH - UPSIZE TOPPER (ONLY), VHO DRINK - CUP REGULAR 24 OZ FOUNTAIN, VHO DRINK - CUP KIDDIE FOUNTAIN, VHO CNDY - M&M MINI, VHO CORN - KIDDIE TRAY (SANS DRINK), VHO CORN - POPCORN REGULAR 130oz.)",6
2237,0.001022,"(VHO DRINK - CUP REGULAR 24 OZ FOUNTAIN, VHO DRINK - CUP KIDDIE FOUNTAIN, VHO CNDY - M&M MINI, VHO CORN - POPCORN SMALL 85 OZ, VHO CORN - KIDDIE TRAY (SANS DRINK), VHO MERCH - PROMO CUP KIDDIE SURCHARGE)",6
2238,0.001431,"(VHO DRINK - CUP REGULAR 24 OZ FOUNTAIN, VHO CORN - BUTTER - REGULAR, VHO CORN - BUTTER - LARGE, VHO DRINK - CUP LARGE 32 OZ FOUNTAIN, VHO CORN - POPCORN LARGE 170 OZ, VHO CORN - POPCORN REGULAR 130oz.)",6
2239,0.001144,"(VHO CORN - BUTTER - LARGE, VHO DRINK - CUP KIDDIE FOUNTAIN, VHO DRINK - CUP LARGE 32 OZ FOUNTAIN, VHO CORN - KIDDIE TRAY (SANS DRINK), VHO MERCH - PROMO CUP KIDDIE SURCHARGE, VHO CORN - POPCORN LARGE 170 OZ)",6
2240,0.001349,"(VHO DRINK - CUP REGULAR 24 OZ FOUNTAIN, VHO DRINK - CUP KIDDIE FOUNTAIN, VHO CORN - BUTTER - REGULAR, VHO CORN - KIDDIE TRAY (SANS DRINK), VHO MERCH - PROMO CUP KIDDIE SURCHARGE, VHO CORN - POPCORN REGULAR 130oz.)",6
2241,0.001104,"(VHO CORN - BUTTER - LARGE, VHO DRINK - CUP KIDDIE FOUNTAIN, VHO CNDY - M&M MINI, VHO DRINK - CUP LARGE 32 OZ FOUNTAIN, VHO CORN - KIDDIE TRAY (SANS DRINK), VHO MERCH - PROMO CUP KIDDIE SURCHARGE, VHO CORN - POPCORN LARGE 170 OZ)",7
2242,0.001349,"(VHO DRINK - CUP REGULAR 24 OZ FOUNTAIN, VHO DRINK - CUP KIDDIE FOUNTAIN, VHO CNDY - M&M MINI, VHO CORN - BUTTER - REGULAR, VHO CORN - KIDDIE TRAY (SANS DRINK), VHO MERCH - PROMO CUP KIDDIE SURCHARGE, VHO CORN - POPCORN REGULAR 130oz.)",7


In [12]:
display(frequent_itemsets.sort_values(by=['support'], ascending=False).head())

,support,itemsets,length
94,0.281656,(VHO DRINK - CUP REGULAR 24 OZ FOUNTAIN),1
66,0.275689,(VHO CORN - POPCORN LARGE 170 OZ),1
67,0.219366,(VHO CORN - POPCORN REGULAR 130oz.),1
93,0.218630,(VHO DRINK - CUP LARGE 32 OZ FOUNTAIN),1
59,0.133246,(VHO CORN - BUTTER - LARGE),1


In [13]:
sub = frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.02) ]
display(sub)

,support,itemsets,length
203,0.021540,"(VHO AVX - Reserve Seating Fee, VHO CORN - POPCORN LARGE 170 OZ)",2
214,0.022848,"(VHO DRINK - CUP REGULAR 24 OZ FOUNTAIN, VHO AVX - Reserve Seating Fee)",2
300,0.026077,"(VHO CNDY - M&M CHOCOLATE, VHO CORN - POPCORN LARGE 170 OZ)",2
308,0.021826,"(VHO CNDY - M&M CHOCOLATE, VHO DRINK - CUP REGULAR 24 OZ FOUNTAIN)",2
324,0.056364,"(VHO CNDY - M&M MINI, VHO CORN - KIDDIE TRAY (SANS DRINK))",2
332,0.053993,"(VHO CNDY - M&M MINI, VHO DRINK - CUP KIDDIE FOUNTAIN)",2
338,0.022031,"(VHO CNDY - M&M MINI, VHO MERCH - PROMO CUP KIDDIE SURCHARGE)",2
347,0.032167,"(VHO CNDY - M&M PEANUT, VHO CORN - POPCORN LARGE 170 OZ)",2
355,0.021254,"(VHO CNDY - M&M PEANUT, VHO DRINK - CUP LARGE 32 OZ FOUNTAIN)",2
356,0.028570,"(VHO DRINK - CUP REGULAR 24 OZ FOUNTAIN, VHO CNDY - M&M PEANUT)",2


In [14]:
%time rules = association_rules(frequent_itemsets, min_threshold=0.1)

Wall time: 132 ms


In [15]:
# Add the length of the antecedent (left hand side)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))

rules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9712 entries, 0 to 9711
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   antecedents         9712 non-null   object 
 1   consequents         9712 non-null   object 
 2   antecedent support  9712 non-null   float64
 3   consequent support  9712 non-null   float64
 4   support             9712 non-null   float64
 5   confidence          9712 non-null   float64
 6   lift                9712 non-null   float64
 7   leverage            9712 non-null   float64
 8   conviction          9712 non-null   float64
 9   antecedent_len      9712 non-null   int64  
dtypes: float64(7), int64(1), object(2)
memory usage: 758.9+ KB


In [16]:
display(rules.head(10))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(VHO CNDY - M&M CARAMEL),(VHO AVX - Reserve Seating Fee),0.012630,0.119104,0.001431,0.113269,0.951005,-0.000074,0.993419,1
1,(VHO AVX - Reserve Seating Fee),(VHO CORN - POPCORN LARGE 170 OZ),0.119104,0.275689,0.021540,0.180851,0.655997,-0.011296,0.884224,1
2,(VHO AVX - Reserve Seating Fee),(VHO CORN - POPCORN REGULAR 130oz.),0.119104,0.219366,0.018311,0.153741,0.700842,-0.007816,0.922453,1
3,(VHO AVX - Reserve Seating Fee),(VHO DRINK - CUP LARGE 32 OZ FOUNTAIN),0.119104,0.218630,0.019619,0.164722,0.753429,-0.006421,0.935461,1
4,(VHO AVX - Reserve Seating Fee),(VHO DRINK - CUP REGULAR 24 OZ FOUNTAIN),0.119104,0.281656,0.022848,0.191833,0.681088,-0.010698,0.888855,1
5,(VHO DRINK - UPSIZE LRG FOUNTAIN),(VHO AVX - Reserve Seating Fee),0.036990,0.119104,0.004210,0.113812,0.955569,-0.000196,0.994028,1
6,(VHO MERCH - UPSIZE 44 OZ FTN PROMO W/TOPPER),(VHO AVX - Reserve Seating Fee),0.014469,0.119104,0.001635,0.112994,0.948703,-0.000088,0.993112,1
7,(VHO OT - DOG HOTDOG CLASSIC),(VHO AVX - Reserve Seating Fee),0.023379,0.119104,0.002412,0.103147,0.866023,-0.000373,0.982208,1
8,(VHO CBO - (5) DOG N' NACHO (LG FTN)...),(VHO CORN - POPCORN LARGE 170 OZ),0.012385,0.275689,0.001839,0.148515,0.538705,-0.001575,0.850645,1
9,(VHO CBO - (5) DOG N' NACHO (LG FTN)...),(VHO CORN - POPCORN REGULAR 130oz.),0.012385,0.219366,0.001349,0.108911,0.496481,-0.001368,0.876045,1


## Exploring the Rules

This is where human creativity and "exploration" comes in.

In [17]:
# Find all rules that have at least 3 items on the LHS, confidence of at least .75 and lift of at least 1.2; sort by support

sub = rules[ (rules['antecedent_len'] >= 3) &
       (rules['confidence'] > 0.75) &
       (rules['lift'] > 1.2) ].sort_values(by="support", ascending=False)
sub.shape
display(sub.head(10))

(1529, 10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
3732,"(VHO CNDY - M&M MINI, VHO MERCH - PROMO CUP KIDDIE SURCHARGE, VHO DRINK - CUP KIDDIE FOUNTAIN)",(VHO CORN - KIDDIE TRAY (SANS DRINK)),0.021213,0.058612,0.021213,1.000000,17.061367,0.019970,inf,3
3729,"(VHO CNDY - M&M MINI, VHO CORN - KIDDIE TRAY (SANS DRINK), VHO MERCH - PROMO CUP KIDDIE SURCHARGE)",(VHO DRINK - CUP KIDDIE FOUNTAIN),0.022031,0.056119,0.021213,0.962894,17.158172,0.019977,25.437601,3
3731,"(VHO CORN - KIDDIE TRAY (SANS DRINK), VHO MERCH - PROMO CUP KIDDIE SURCHARGE, VHO DRINK - CUP KIDDIE FOUNTAIN)",(VHO CNDY - M&M MINI),0.021949,0.056609,0.021213,0.966480,17.072860,0.019971,28.144493,3
3704,"(VHO CNDY - M&M MINI, VHO CORN - KIDDIE TRAY (SANS DRINK), VHO DRINK - CUP REGULAR 24 OZ FOUNTAIN)",(VHO DRINK - CUP KIDDIE FOUNTAIN),0.014714,0.056119,0.014060,0.955556,17.027401,0.013235,21.237329,3
3706,"(VHO CNDY - M&M MINI, VHO DRINK - CUP REGULAR 24 OZ FOUNTAIN, VHO DRINK - CUP KIDDIE FOUNTAIN)",(VHO CORN - KIDDIE TRAY (SANS DRINK)),0.014060,0.058612,0.014060,1.000000,17.061367,0.013236,inf,3
3707,"(VHO DRINK - CUP REGULAR 24 OZ FOUNTAIN, VHO CORN - KIDDIE TRAY (SANS DRINK), VHO DRINK - CUP KIDDIE FOUNTAIN)",(VHO CNDY - M&M MINI),0.014673,0.056609,0.014060,0.958217,16.926891,0.013230,22.578487,3
3512,"(VHO CNDY - M&M MINI, VHO CORN - KIDDIE TRAY (SANS DRINK), VHO CORN - POPCORN LARGE 170 OZ)",(VHO DRINK - CUP KIDDIE FOUNTAIN),0.012385,0.056119,0.011690,0.943894,16.819607,0.010995,16.823296,3
3514,"(VHO CNDY - M&M MINI, VHO CORN - POPCORN LARGE 170 OZ, VHO DRINK - CUP KIDDIE FOUNTAIN)",(VHO CORN - KIDDIE TRAY (SANS DRINK)),0.011690,0.058612,0.011690,1.000000,17.061367,0.011005,inf,3
3515,"(VHO CORN - KIDDIE TRAY (SANS DRINK), VHO CORN - POPCORN LARGE 170 OZ, VHO DRINK - CUP KIDDIE FOUNTAIN)",(VHO CNDY - M&M MINI),0.012017,0.056609,0.011690,0.972789,17.184302,0.011009,34.669613,3
3575,"(VHO CORN - KIDDIE TRAY (SANS DRINK), VHO DRINK - CUP KIDDIE FOUNTAIN, VHO CORN - POPCORN REGULAR 130oz.)",(VHO CNDY - M&M MINI),0.010177,0.056609,0.009728,0.955823,16.884601,0.009152,21.354938,3


In [18]:
# mlxtend stores the itemsets as frozensets. IF we want to filter rules by what items are in them, 
# we have to check for set membership.

# This little helper will check if any elements in set1 are in set2
def is_any_in(set1, set2):
    c = set1.intersection(set2)
    return len(c) != 0

# Testing out our little helper function to make sure it works!
setb = frozenset({'cheese', 'soda', 'milk', 'ham'})
is_any_in(set({'cheese'}), setb) # Expect True
is_any_in(set({'cheese', 'ice cream'}), setb) # Expect True
is_any_in(set({'legos', 'ice cream'}), setb) # Expect False

True

True

False

In [19]:
# Just as a reminder: what items are in the Large Popcorn category?
items[items['US_Is_LrgPopcorn'] == 1]

,CINEPLEX_Item_ID,CINEPLEX_Item_Description,CINEPLEX_Item_Status,EIS_Group_Description,EIS_Sub_Group_Description,Item_Report_Code2,Item_Report_Code3,Item_Class_Description1,Item_Class_Description2,Item_Class_Description3,Is_Item_Package,Is_Box_Office_Item,US_Is_Butter,US_Is_Kiddietray,US_Is_Voucher,US_Is_LrgPopcorn,US_Is_Coffee,US_Is_Candy,US_Is_Alcohol,US_Is_MenuFood
333,93845,VHO CORN - POPCORN LARGE 170 OZ,A,Popcorn,Popcorn,880 Cals,Y,VHO-POPCORN,0,0,0,0,0,0,0,1,0,0,0,0
334,93913,VHO CORN - VIP POPCORN LARGE 170oz,A,0,0,880 Cals,0,VHO-POPCORN,0,0,0,0,0,0,0,1,0,0,0,0
350,93856,VHO CORN - UPSIZE TO LARGE CORN,A,Popcorn,Popcorn,Add 290 Cals,0,VHO-POPCORN,0,0,0,0,0,0,0,1,0,0,0,0


In [20]:
# Let's find all rules that contain any of the popcorn items

search_set = set({'VHO CORN - POPCORN LARGE 170 OZ', 'VHO CORN - VIP POPCORN LARGE 170oz', 'VHO CORN - UPSIZE TO LARGE CORN'})

sub = rules[(rules['antecedents'].apply(lambda x: is_any_in(search_set, x))) & 
            (rules['antecedent_len'] >=2) ].sort_values('lift', ascending=False)

sub.shape
display(sub.head(10))

(1999, 10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
8057,"(VHO VIP - Reserve Seating Fee, VHO CORN - VIP POPCORN LARGE 170oz)","(VHO VIP Surcharge, VHO DRINK - VIP DRINK LARGE 32oz, VHO CORN - BUTTER - LARGE)",0.005477,0.001635,0.001594,0.291045,178.017537,0.001585,1.408220,2
8052,"(VHO VIP Surcharge, VHO CORN - VIP POPCORN LARGE 170oz)","(VHO VIP - Reserve Seating Fee, VHO DRINK - VIP DRINK LARGE 32oz, VHO CORN - BUTTER - LARGE)",0.005559,0.001635,0.001594,0.286765,175.399632,0.001585,1.399770,2
7820,"(VHO CORN - BUTTER - SMALL, VHO CORN - POPCORN LARGE 170 OZ)","(VHO CORN - POPCORN SMALL 85 OZ, VHO CORN - BUTTER - LARGE, VHO DRINK - CUP LARGE 32 OZ FOUNTAIN)",0.003229,0.002044,0.001144,0.354430,173.429873,0.001138,1.545854,2
7814,"(VHO CORN - BUTTER - SMALL, VHO DRINK - CUP LARGE 32 OZ FOUNTAIN, VHO CORN - POPCORN LARGE 170 OZ)","(VHO CORN - POPCORN SMALL 85 OZ, VHO CORN - BUTTER - LARGE)",0.001553,0.004455,0.001144,0.736842,165.390633,0.001138,3.783070,3
4757,"(VHO CORN - BUTTER - SMALL, VHO CORN - POPCORN LARGE 170 OZ)","(VHO CORN - POPCORN SMALL 85 OZ, VHO CORN - BUTTER - LARGE)",0.003229,0.004455,0.002207,0.683544,153.427476,0.002193,3.145922,2
5079,"(VHO VIP Surcharge, VHO CORN - VIP POPCORN LARGE 170oz)","(VHO VIP - Reserve Seating Fee, VHO CORN - BUTTER - LARGE)",0.005559,0.004619,0.003597,0.647059,140.096825,0.003571,2.820247,2
5077,"(VHO VIP - Reserve Seating Fee, VHO CORN - VIP POPCORN LARGE 170oz)","(VHO VIP Surcharge, VHO CORN - BUTTER - LARGE)",0.005477,0.004782,0.003597,0.656716,137.326700,0.003571,2.899113,2
8042,"(VHO VIP Surcharge, VHO DRINK - VIP DRINK LARGE 32oz, VHO CORN - VIP POPCORN LARGE 170oz)","(VHO VIP - Reserve Seating Fee, VHO CORN - BUTTER - LARGE)",0.002575,0.004619,0.001594,0.619048,134.032027,0.001582,2.612876,3
8047,"(VHO VIP - Reserve Seating Fee, VHO DRINK - VIP DRINK LARGE 32oz, VHO CORN - VIP POPCORN LARGE 170oz)","(VHO VIP Surcharge, VHO CORN - BUTTER - LARGE)",0.002575,0.004782,0.001594,0.619048,129.449735,0.001582,2.612447,3
7822,"(VHO CORN - POPCORN SMALL 85 OZ, VHO CORN - POPCORN LARGE 170 OZ)","(VHO CORN - BUTTER - SMALL, VHO CORN - BUTTER - LARGE, VHO DRINK - CUP LARGE 32 OZ FOUNTAIN)",0.007112,0.001308,0.001144,0.160920,123.033046,0.001135,1.190222,2
